steps:
- clip [x]
- send to rolf 
- need to work on a more permanent solution [x] just use the ML for fusion

In [20]:
import os 
from glob import glob

In [21]:
#gdemH_fn = '/media/ljp238/12TBWolf/RSPROX/OUT_TILES/BLOCKS/TLS/gdemH.tif'
tdemv_filesi = glob("/media/ljp238/12TBWolf/RSPROX/OUT_TILES/TILES12/*/*tdem_DEM__Fw.tif")
tilenames_tls = [
 'N11E104',
 'N11E105',
 'N12E103',
 'N12E104',
 'N12E105',
 'N13E103',
 'N13E104',
 'N13E105']
roi = 'TLS'
block_tiles_dpath = f"/media/ljp238/12TBWolf/RSPROX/OUT_TILES/BLOCKS/{roi}/tiles"
os.makedirs(block_tiles_dpath, exist_ok=True)

tdemv_files = [i for i in tdemv_filesi for t in tilenames_tls if t in i]

In [22]:
import os
import rasterio
from rasterio.mask import mask
from shapely.geometry import box
from concurrent.futures import ProcessPoolExecutor


def get_tile_name(tdemv_path):
    """Extract tile name from the directory structure."""
    return os.path.basename(os.path.dirname(tdemv_path))


def clip_raster(gdemH_fn, tdemv, output_dir):
    """Clip a large raster using a single reference raster and save the output."""
    try:
        with rasterio.open(gdemH_fn) as src, rasterio.open(tdemv) as tdemv_src:
            clip_bounds = tdemv_src.bounds  # Get bounding box
            geom = [box(*clip_bounds)]  # Convert to a polygon

            out_image, out_transform = mask(src, geom, crop=True)
            out_meta = src.meta.copy()
            out_meta.update({
                "driver": "GTiff",
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform
            })

            tilename = get_tile_name(tdemv)
            output_fn = os.path.join(output_dir, f"{tilename}_DTM_GV4.tif")

            with rasterio.open(output_fn, "w", **out_meta) as dest:
                dest.write(out_image)

            print(f"Saved: {output_fn}")
            return output_fn

    except Exception as e:
        print(f"Error processing {tdemv}: {e}")
        return None


def process_clipping_parallel(gdemH_fn, tdemv_files, output_dir, num_workers=4):
    """Parallelized processing of raster clipping."""
    os.makedirs(output_dir, exist_ok=True)

    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        futures = {executor.submit(clip_raster, gdemH_fn, tdemv, output_dir): tdemv for tdemv in tdemv_files}

        for future in futures:
            future.result()  # Wait for completion


In [28]:
tls_dtm_fn = '/media/ljp238/12TBWolf/RSPROX/OUT_TILES/BLOCKS/TLS/DOWNX/DTM/TLS_DTM_H.tif'
tls_dsm_fn = '/media/ljp238/12TBWolf/RSPROX/OUT_TILES/BLOCKS/TLS/DOWNX/DSM/TLS_DTM_H.tif'

In [29]:
dtm_tile_dpath = '/media/ljp238/12TBWolf/RSPROX/OUT_TILES/BLOCKS/TLS/DOWNX/DTM/tiles/'
process_clipping_parallel(tls_dtm_fn, tdemv_files, dtm_tile_dpath, num_workers=15)

Saved: /media/ljp238/12TBWolf/RSPROX/OUT_TILES/BLOCKS/TLS/DOWNX/DTM/tiles/N13E105_DTM_GV4.tif
Saved: /media/ljp238/12TBWolf/RSPROX/OUT_TILES/BLOCKS/TLS/DOWNX/DTM/tiles/N13E103_DTM_GV4.tif
Saved: /media/ljp238/12TBWolf/RSPROX/OUT_TILES/BLOCKS/TLS/DOWNX/DTM/tiles/N13E104_DTM_GV4.tif
Saved: /media/ljp238/12TBWolf/RSPROX/OUT_TILES/BLOCKS/TLS/DOWNX/DTM/tiles/N12E104_DTM_GV4.tif
Saved: /media/ljp238/12TBWolf/RSPROX/OUT_TILES/BLOCKS/TLS/DOWNX/DTM/tiles/N11E105_DTM_GV4.tif
Saved: /media/ljp238/12TBWolf/RSPROX/OUT_TILES/BLOCKS/TLS/DOWNX/DTM/tiles/N12E103_DTM_GV4.tif
Saved: /media/ljp238/12TBWolf/RSPROX/OUT_TILES/BLOCKS/TLS/DOWNX/DTM/tiles/N12E105_DTM_GV4.tif
Saved: /media/ljp238/12TBWolf/RSPROX/OUT_TILES/BLOCKS/TLS/DOWNX/DTM/tiles/N11E104_DTM_GV4.tif


In [30]:
dsm_tile_dpath = "/media/ljp238/12TBWolf/RSPROX/OUT_TILES/BLOCKS/TLS/DOWNX/DSM/tiles"
process_clipping_parallel(tls_dsm_fn, tdemv_files, dsm_tile_dpath, num_workers=15)

Saved: /media/ljp238/12TBWolf/RSPROX/OUT_TILES/BLOCKS/TLS/DOWNX/DSM/tiles/N13E105_DTM_GV4.tif
Saved: /media/ljp238/12TBWolf/RSPROX/OUT_TILES/BLOCKS/TLS/DOWNX/DSM/tiles/N13E103_DTM_GV4.tif
Saved: /media/ljp238/12TBWolf/RSPROX/OUT_TILES/BLOCKS/TLS/DOWNX/DSM/tiles/N12E104_DTM_GV4.tif
Saved: /media/ljp238/12TBWolf/RSPROX/OUT_TILES/BLOCKS/TLS/DOWNX/DSM/tiles/N12E103_DTM_GV4.tif
Saved: /media/ljp238/12TBWolf/RSPROX/OUT_TILES/BLOCKS/TLS/DOWNX/DSM/tiles/N12E105_DTM_GV4.tif
Saved: /media/ljp238/12TBWolf/RSPROX/OUT_TILES/BLOCKS/TLS/DOWNX/DSM/tiles/N13E104_DTM_GV4.tif
Saved: /media/ljp238/12TBWolf/RSPROX/OUT_TILES/BLOCKS/TLS/DOWNX/DSM/tiles/N11E104_DTM_GV4.tif
Saved: /media/ljp238/12TBWolf/RSPROX/OUT_TILES/BLOCKS/TLS/DOWNX/DSM/tiles/N11E105_DTM_GV4.tif


In [32]:
# work with DSMh
#dsm_tile_dpath = "/media/ljp238/12TBWolf/RSPROX/OUT_TILES/BLOCKS/TLS/DOWNX/DSM/tiles2"
#process_clipping_parallel(tls_dsm_fn, tdemv_files, dsm_tile_dpath, num_workers=15)

In [23]:
#process_clipping_parallel(gdemH_fn, tdemv_files, block_tiles_dpath, num_workers=10)

In [17]:
import subprocess

def list2txt(txt_file,filelist):
    with open(txt_file, 'w') as f:
        for tif in filelist:
            f.write(f"{tif}\n")
    print('list2txt')
    
def build_vrt(filelist,vrt_file):

    # g
    txt_file = vrt_file.replace('.vrt', '.txt')
   # gpkg_file = vrt_file.replace('.vrt', '.gpkg')

    if not os.path.isfile(vrt_file):
        list2txt(txt_file,filelist)
        subprocess.run(['gdalbuildvrt', '-input_file_list', txt_file, vrt_file])

    print(f"VRT created: {vrt_file}")

In [19]:
files = glob("/media/ljp238/12TBWolf/RSPROX/OUT_TILES/BLOCKS/TLS/tiles/*.tif")
vrt = '/media/ljp238/12TBWolf/RSPROX/OUT_TILES/BLOCKS/TLS/GV4.vrt'
build_vrt(files,vrt)

list2txt
0...10...20...30...40...50...60...70...80...90...100 - done.
VRT created: /media/ljp238/12TBWolf/RSPROX/OUT_TILES/BLOCKS/TLS/GV4.vrt


In [ ]:
gdemH_fn this a big raster, cut it by this files tdemv_files
save as 
tilename = tdemv_files[i].split('/')[-2]
fo = os.path.join(block_tiles_dpath, f'{tilename}_DTM_GV4.tif')
# goal is to clip the data 